# Output data post-processing Water-Energy model

This notebook run the water-energy model and post-processing its results 

In [4]:
from pywr.model              import Model
from pywr.notebook           import draw_graph
from pywr.recorders          import TablesRecorder
from pywr.recorders.progress import ProgressRecorder
from matplotlib              import pyplot            as plt
import tables
import pandas
import seaborn
import numpy                                          as np
import os

import sys
sys.path.append('\\Users\\n65847jg\\Documents\\Doctorado\\Thesis\\Paper_1\\hydraville\\hydraville')
#sys.path.append('\\Users\\joseg\\Documents\\Postgraduate\\Doctorado\\Thesis\\Paper_1\\hydraville\\hydraville')
from moea                    import PyretoJSONPlatypusWrapper


%matplotlib inline

## Model Paths

In [5]:
JSON_PATH   = '../test'
#JSON_PATH   = '../water-energy_without-OR'
#JSON_PATH   = '../water-energyM'
DATA_PATH   = '../data'
OUTPUT_PATH = '../water-energyM/outputs'
FIG_PATH    = os.path.join(OUTPUT_PATH, 'figures')

## Water-Energy model

Here we run the water-energy model and plot some comparison of baseline results

In [6]:
model = Model.load(os.path.join(JSON_PATH,'water-only.json'),)
#model = Model.load(os.path.join(JSON_PATH,'water-energy.json'),)
draw_graph(model)

<IPython.core.display.Javascript object>

In [129]:
with tables.open_file(os.path.join(JSON_PATH,'water-energy_outputs.h5')) as h5:
#with tables.open_file('water-energy_dataframes.h5') as h5: 
#    print(h5)
    tbl = h5.get_node('/time')
    date_index = pandas.to_datetime({k: tbl.col(k) for k in ('year', 'month', 'day')})
    
    data = {}
    for ca in h5.walk_nodes('/', 'CArray'):
#        data[ca._v_name] = pandas.DataFrame(ca.read(), index=date_index, columns=model.scenarios.multiindex)
        data[ca._v_name] = pandas.DataFrame(ca.read(), index=date_index)

#print(data)


#df = pandas.DataFrame(data, index=date_index)
df = pandas.concat(data, axis=1)
#print(df)

nrows = len(df.columns.levels[0])
#df.plot(subplots=True, )

FLOW_UNITS = 'Mm^3'
ENERGY_UNITS = 'MWh' 
PERCENTILES = np.linspace(0, 100)

Y_LABEL_MAP = {
        'catchment1'   : f'Catchment 1 inflow [${FLOW_UNITS}/day$]',
        'catchment2'   : f'Catchment 2 inflow [${FLOW_UNITS}/day$]',
        'catchment3'   : f'Catchment 3 inflow [${FLOW_UNITS}/day$]',
        'catchment4'   : f'Catchment 4 inflow [${FLOW_UNITS}/day$]',
        'catchment5'   : f'Catchment 5 inflow [${FLOW_UNITS}/day$]',
        'irrigation1'  : f'Irrigation Sector 1 [${FLOW_UNITS}/day$]',
        'irrigation2'  : f'Irrigation Sector 2 [${FLOW_UNITS}/day$]',
        'water_supply1': f'Water Supply [${FLOW_UNITS}/day$]',
        'reservoir1'   : f'Reservoir 1 Storage [${FLOW_UNITS}$]',
        'reservoir2'   : f'Reservoir 2 Storage [${FLOW_UNITS}$]',
        'turbine_energy_generation1': f'Hydropower 1 [${ENERGY_UNITS}$]',
        'turbine_energy_generation2': f'Hydropower 2 [${ENERGY_UNITS}$]',
        'thermal1'     : f'Thermal plant 1 [${ENERGY_UNITS}$]',
        'thermal2'     : f'Thermal plant 2 [${ENERGY_UNITS}$]',
        'thermal3'     : f'Thermal plant 3 [${ENERGY_UNITS}$]',
 }

dfvalues  = list(df.columns.levels[0])
dicvalues = list(Y_LABEL_MAP.keys())

values_to_remove = [item for item in dfvalues if item not in dicvalues]

df2 = df.drop(columns=values_to_remove, axis=0, level=0)
#df2.columns = df2.columns.droplevel()

nrows = len(df2.columns.levels[0])

for node in df2.columns.levels[0]:
    fig, ax = plt.subplots(nrows=1, figsize=(12, 4))
    df2[node].plot(ax=ax, color='grey', alpha=0.5, legend=False)

    try:
        label = Y_LABEL_MAP[node]
    except KeyError:
        label = node
    
    ax.set_ylabel(label)
    plt.tight_layout()
    fig.savefig(os.path.join(FIG_PATH, f'{node}.png'), dpi=300)
#    fig.savefig(os.path.join(FIG_PATH, f'{node}.eps'))


['bus1', 'bus2', 'catchment1', 'catchment1_inflow', 'catchment2', 'catchment2_inflow', 'catchment3', 'catchment3_inflow', 'catchment4', 'catchment4_inflow', 'catchment5', 'catchment5_inflow', 'channel1_capacity', 'channel2_capacity', 'channel3_capacity', 'channel4_capacity', 'energy1_demand', 'energy2_demand', 'energy_demand1', 'energy_demand2', 'hydropower_turbine1', 'hydropower_turbine2', 'irrigation1', 'irrigation1_channel1', 'irrigation1_channel2', 'irrigation1_demand', 'irrigation2', 'irrigation2_channel3', 'irrigation2_demand', 'reservoir1', 'reservoir1_capacity', 'reservoir2', 'reservoir2_capacity', 'return_flow1', 'return_flow2', 'river1', 'river2', 'river3', 'river4', 'spill1', 'spill2', 'term1', 'thermal1', 'thermal1_capacity', 'thermal2', 'thermal2_capacity', 'thermal3', 'thermal3_capacity', 'transmission_capacity', 'transmission_line', 'turbine1', 'turbine1_above', 'turbine1_below', 'turbine1_capacity', 'turbine1_control', 'turbine2', 'turbine2_above', 'turbine2_below', 'tu

"\nfor node in df.columns.levels[0]:\n    fig, ax = plt.subplots(nrows=1, figsize=(12, 4))\n    df[node].plot(ax=ax, color='grey', alpha=0.5, legend=False)\n\n    try:\n        label = Y_LABEL_MAP[node]\n    except KeyError:\n        label = node\n    \n    ax.set_ylabel(label)\n    plt.tight_layout()\n    fig.savefig(os.path.join(FIG_PATH, f'{node}.png'), dpi=300)\n#    fig.savefig(os.path.join(FIG_PATH, f'{node}.eps'))\n\n"

In [140]:
#with tables.open_file(os.path.join(DATA_PATH,'catchmod_flows_by_catchment_sub010.h5')) as Input:
#with tables.open_file(os.path.join(JSON_PATH,'water-energy_outputs.h5')) as Input:
#with tables.open_file(os.path.join(DATA_PATH,'flow.h5')) as Input:
#    print(Input)